In [ ]:
https://docs.osmcode.org/pyosmium/latest/ref_index.html
https://osmcode.org/osmium-tool/manual.html
https://docs.osmcode.org/osmium/latest/

In [18]:
import osmium as osm
import pandas as pd
from overpass import API

api = API()

#get starting lat and lon from the phone GPS
starting_lat = 47.5846209
starting_lon = -122.3010940
roads_response = api.get('way["highway" ="residential"](around:50,{0},{1});'.format(starting_lat,starting_lon)) 
buildings_response = api.get('way["building"="yes"](around:50,{0},{1});'.format(starting_lat,starting_lon)) 
trees_response = api.get('node["natural"="tree"](around:50,{0},{1});'.format(starting_lat,starting_lon)) 

roads = []
buildings = []
trees = []

for way in roads_response.features:
    roads.append(way.id)
for building in buildings_response.features:
    buildings.append(building.id)
for tree in trees_response.features:
    trees.append(tree.id)

class OSMHandler(osm.SimpleHandler):
    
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        # self.address = {'addr:housenumber': '2400', 'addr:postcode': '98144', 'addr:street': 'South Hill Street'}
        self.roads_with_latlon = []
        self.buildings_with_latlon = []
        self.trees_with_latlon = []
    def node(self, n):
        if n.id in trees:
            self.trees_with_latlon.append(n)

    def way(self, w):
       
        if w.id in roads:
            list = []
            for road in w.nodes:
                list.append(road)
            self.roads_with_latlon.append(list)
        if w.id in buildings:
            list = []
            for building in w.nodes:
                list.append(building)
            self.buildings_with_latlon.append(list)
            
    # def relation(self, r):
    #     self.tag_inventory(r, "relation")

    
osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file("../Downloads/smaller_map.osm", locations=True, idx='flex_mem')

# transform the list into a pandas DataFrame
# data_colnames = ['type', 'id', 'version', 'visible', 'ts', 'uid',
#                  'user', 'chgset', 'ntags', 'tagkey', 'tagvalue']
# df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
# osmhandler.roads_with_latlon


ConnectionError: HTTPSConnectionPool(host='overpass-api.de', port=443): Max retries exceeded with url: /api/interpreter (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f9c3e3f26d8>: Failed to resolve 'overpass-api.de' ([Errno -3] Temporary failure in name resolution)"))

In [19]:
osmhandler.roads_with_latlon 

[[osmium.osm.NodeRef(ref=53136415, location=osmium.osm.Location(x=-1223015067, y=475869198)),
  osmium.osm.NodeRef(ref=5757431543, location=osmium.osm.Location(x=-1223015731, y=475868612)),
  osmium.osm.NodeRef(ref=2516369689, location=osmium.osm.Location(x=-1223015896, y=475868466)),
  osmium.osm.NodeRef(ref=152010665, location=osmium.osm.Location(x=-1223016159, y=475868093)),
  osmium.osm.NodeRef(ref=2516441880, location=osmium.osm.Location(x=-1223016311, y=475867646)),
  osmium.osm.NodeRef(ref=9193214107, location=osmium.osm.Location(x=-1223016316, y=475866142)),
  osmium.osm.NodeRef(ref=5440359896, location=osmium.osm.Location(x=-1223016323, y=475864945)),
  osmium.osm.NodeRef(ref=4799960441, location=osmium.osm.Location(x=-1223016328, y=475863911)),
  osmium.osm.NodeRef(ref=6632334769, location=osmium.osm.Location(x=-1223016329, y=475863758)),
  osmium.osm.NodeRef(ref=53138092, location=osmium.osm.Location(x=-1223016332, y=475863228)),
  osmium.osm.NodeRef(ref=4799960440, location

In [20]:
class RoadNode:
    def __init__(self, r):
        # self.visited = 0
        self.nodes = []
        self.intersection = 0
        if r != None:
            self.ref = r.ref
            self.lat = r.location.lat
            self.lon = r.location.lon
        else:
            self.ref = None
            self.lat = None
            self.lon = None
    def addNode(self, node):
        self.nodes = self.nodes + [node]


road_nodes = {}
starting_street_segment = None
length = 0

for roadds in osmhandler.roads_with_latlon:
    prev_node = None

    for road in roadds:
        y = abs(road.location.lat - starting_lat)
        x = abs(road.location.lon - starting_lon)
        distance = x + y
        if starting_street_segment == None:
            starting_street_segment = [distance, road]
        elif starting_street_segment[0] > distance:
            starting_street_segment = [distance, road]
            
        #inserting to road_nodes dict.
        if prev_node == None:
            if road.ref in road_nodes:
                prev_node = road_nodes[road.ref]
            else:
                node = RoadNode(road) 
                prev_node = node
                road_nodes[road.ref] = node
            
        else:
            length = length + 1
            
            if road.ref in road_nodes:
                node = road_nodes[road.ref]
            else:
                node = RoadNode(road)      
                road_nodes[road.ref] = node
            
            
            node.addNode(prev_node)
            
            prev_node.addNode(node)
            
            prev_node = node

for key in road_nodes:
    if len(road_nodes[key].nodes) > 2:
        road_nodes[key].intersection = 1



        

In [21]:
print(starting_street_segment)

[0.0002193999999988705, osmium.osm.NodeRef(ref=3663412746, location=osmium.osm.Location(x=-1223009966, y=475844989))]


In [32]:
# def fly_drone_to_dest(start, dest, visited):
#     start = 0
#     end = 0
#     for i, r in enumerate(visited):
#         if r == start:
#             start = i
#         elif r == dest:
#             end = i
#     path = []
#     if start > end:
#         for i in range(start, end, -1):
#             path.append(visited[i])
#     else:
#         path = visited[start:end]
#     return path

def instructions_dfs(roadNode, visited, intersection_point):
    
    visited = visited + [roadNode]

    if roadNode.intersection == 1:
        intersection = roadNode
    else:
        intersection = intersection_point
    
    for r in roadNode.nodes:
        if r in visited:
            continue

        
        visited = instructions_dfs(r, visited, intersection)
        
        if intersection_point != None or roadNode.intersection == 1:
            visited = visited + [roadNode]

    return visited
     
all_dir = []

# for key in road_nodes: 
#     
#     if road_nodes[key] in visited: 
#         continue
#     # visited = visited + [road_nodes[key]]
#     list = instructions_dfs(road_nodes[key], visited, road_nodes[key])
#     all_dir.append(list)

# visited = []
# for node in road_nodes[starting_point[1].ref].nodes:
#     print(node.ref)
#     print("~~~")
#     list = instructions_dfs(road_nodes[starting_point[1].ref], visited, road_nodes[starting_point[1].ref])
#     for r in list:
#         print(r.ref)
#     print("----")
#     if len(list) > length:
#         all_dir = all_dir + [list]

list = instructions_dfs(road_nodes[starting_street_segment[1].ref],  [], road_nodes[starting_street_segment[1].ref])
all_dir = all_dir + [list]
for r in all_dir:
    for n in r:
        print(n.ref)

3663412746
4799922590
53232306
4799922593
8416045670
4799922611
53148758
6632334766
4799922610
8416045669
4799960440
53138092
6632334769
4799960441
5440359896
9193214107
2516441880
152010665
2516369689
5757431543
53136415
5757431543
2516369689
152010665
2516441880
9193214107
5440359896
4799960441
6632334769
53138092
4799960440
8416045669
4799922610
6632334766
53148758
4799922611
8416045670
4799922593
53232306
4799922587
4806589034
53150019
4806589034
4799922587
53232306
4799922588
5440360055
3663412752
8374179171
3496255251
8374179171
3663412752
5440360055
4799922588
53232306
4799922590
3663412746
4799890424
53202700
4799890424
3663412746


In [33]:
import numpy as np

smallest_route = None
for route in all_dir:
    if smallest_route == None:
        smallest_route = route
    elif len(smallest_route) > len(route):
        smallest_route = route

start_point = smallest_route.pop(0)

vector_smallest_route = []
for route in smallest_route:
    x_comp = (start_point.lon - route.lon)*100000
    y_comp = (start_point.lat - route.lat)*100000
    dist = np.linalg.norm(np.array([x_comp, y_comp]) -np.array([0, 0]))
    
    start_point = route
    vector_smallest_route.append([ x_comp, y_comp, dist])


In [37]:
https://stackoverflow.com/questions/35935670/share-variables-between-different-jupyter-notebooks

SyntaxError: invalid syntax (3776508263.py, line 1)

In [34]:
%store vector_smallest_route

Stored 'vector_smallest_route' (list)


In [35]:
%store

Stored variables and their in-db values:
vector_smallest_route             -> [[52.38999999903626, 0.010000000116860974, 52.3900


In [36]:
vector_smallest_route

[[52.38999999903626, 0.010000000116860974, 52.39000095341688],
 [13.750000000811724, -0.5200000003924288, 13.759829214882377],
 [-0.11999999998124622, -8.079999999921483, 8.080891039899416],
 [-0.7800000005886432, -42.849999999816646, 42.85709859504263],
 [-0.5200000003924288, -42.849999999816646, 42.853155076198234],
 [-0.15999999902760464, -5.090000000507189, 5.092514114349808],
 [0.10999999915384251, -5.099999999913507, 5.101186136471753],
 [0.07000000010748408, -3.6199999996711085, 3.620676731998298],
 [-0.5000000001587068, -31.950000000335876, 31.95391212389527],
 [-0.4899999993313031, -37.40000000007626, 37.403209755381276],
 [-0.09000000034120603, -4.939999999464817, 4.940819769509288],
 [-0.029999999640040187, -5.300000000119098, 5.300084905097355],
 [-0.01000000082740371, -1.5300000001161607, 1.5300326795111272],
 [-0.049999999873762135, -10.340000000041982, 10.34012088908324],
 [-0.07000000010748408, -11.969999999905667, 11.970204676518975],
 [-0.049999999873762135, -15.04000

In [13]:
vector_smallest_route[0][:2]

[-0.11999999998124622, -8.079999999921483]